In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from common.constants import CV_DATASETS_DIR

%matplotlib inline 

# Set a seed to make results are consistent
np.random.seed(3)

In [163]:
def layer_sizes(X: np.ndarray, Y: np.ndarray):
    """
    Returns the size of each layer of the neural network
    :param X: input dataset of shape (n, m)
    :param Y: labels of shape (k, m)
    :return: sizes of each layer of the neural network
    """
    n_x = X.shape[0]
    n_y = Y.shape[0]
    return n_x, n_y

def initialize_parameters(n_x: int, n_y: int):
    """
    Initialize the parameters of the neural network
    :param n_x: the size input layer
    :param n_y: the size of output layer
    :return: parameters of the neural network (W - weights matrix of shape (n_x, n_y), b - bias vector)
    """
    W = np.random.randn(n_y, n_x) * 0.01
    b = np.zeros((n_y, 1))
    return {"W": W, "b": b}

def forward_propagate(X: np.ndarray, parameters: dict):
    """
    Perform forward propagation of the neural network
    :param X: input data of shape (n_x, m) 
    :param parameters: parameters of the neural network
    :return: the predicted values of shape (n_x, m)
    """
    W = parameters["W"]
    b = parameters["b"]
    Z = W @ X + b
    Y_hat = Z
    return Y_hat

def compute_cost(Y_hat: np.ndarray, Y: np.ndarray):
    """
    Compute the cost function of the neural network
    :param Y_hat: the output of the neural network
    :param Y: the "real" labels
    :return: sum of squared scaled by 1/(2 * m)
    """
    m = Y_hat.shape[1]
    cost = np.sum((Y_hat - Y) ** 2) / (2 * m)
    return cost

def backward_propagate(Y_hat: np.ndarray, X: np.ndarray, Y: np.ndarray):
    """
    Perform backward propagation of the neural network
    :param Y_hat: the output of the neural network
    :param X: input data of shape (n_x, m)
    :param Y: the "real" labels
    :return: gradients of the weights and bias of the neural network
    """
    m = X.shape[1]
    dZ = Y_hat - Y
    dW = 1/m * np.dot(dZ, X.T)
    db = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return {"dW": dW, "db": db}

def update_parameters(parameters: dict, gradients: dict, learning_rate: float = 1.2):
    """
    Update the parameters of the neural network
    :param parameters: current parameters of the neural network
    :param gradients: gradients of the weights and bias of the neural network
    :param learning_rate: learning rate parameters for gradient descent
    :return: new parameters of the neural network
    """
    W = parameters["W"]
    b = parameters["b"]
    dW = gradients["dW"]
    db = gradients["db"]
    W = W - learning_rate * dW
    b = b - learning_rate * db
    return {"W": W, "b": b}

def nn_model(X: np.ndarray, Y: np.ndarray, num_iterations: int = 10, learning_rate: float = 1.2):
    n_x, n_y = layer_sizes(X, Y)
    print(f"input size: {n_x}, output size: {n_y}")
    parameters = initialize_parameters(n_x, n_y)
    
    for n in range(0, num_iterations):
        Y_hat = forward_propagate(X, parameters)
        # print(n, "Y_hat", Y_hat)
        cost = compute_cost(Y_hat, Y)
        # print(n, "cost", cost)
        gradients = backward_propagate(Y_hat, X, Y)
        # print(n, "grads", gradients)
        parameters = update_parameters(parameters, gradients, learning_rate)
        # print(n, "parameters", parameters)
        print(f"Cost after <{n}> iteration: {cost:.6}")
    
    return parameters

def predict(X, Y, parameters: dict, X_pred: np.ndarray):
    W = parameters["W"]
    b = parameters["b"]
    
    if isinstance(X, pd.Series):
        print("0")
        X_mean = np.mean(X)
        X_std = np.std(X)
        X_pred_norm = ((X_pred - X_mean) / X_std).reshape((1, len(X_pred)))
    else:
        X_mean = np.array(X_multi.mean()).reshape((len(X_multi.axes[1]),1))
        X_std = np.array(X_multi.std()).reshape((len(X_multi.axes[1]),1))
        X_pred_norm = ((X_pred_multi - X_mean)/X_std)
   
    Y_pred_norm = W @ X_pred_norm + b
    Y_pred = Y_pred_norm * np.std(Y) + np.mean(Y)
    return Y_pred

In [ ]:
DATA_PATH1 = Path(CV_DATASETS_DIR, "csv", "tvmarketing.csv")
adv = pd.read_csv(DATA_PATH1)
print(adv.head())
adv.plot(x="TV", y="Sales", kind="scatter", c="black");

In [ ]:
# Do a column-wise normalization of dataset
adv_norm = (adv - adv.mean()) / adv.std()
adv_norm.plot(x="TV", y="Sales", kind="scatter", c="black");

In [ ]:
X_norm = adv_norm["TV"]
Y_norm = adv_norm["Sales"]
X_norm = np.array(X_norm).reshape((1, len(X_norm)))
Y_norm = np.array(Y_norm).reshape((1, len(Y_norm)))

print(f"The shape of X_norm: {X_norm.shape}")
print(f"The shape of Y_norm: {Y_norm.shape}")

In [ ]:
parameters = nn_model(X_norm, Y_norm, num_iterations=10, learning_rate=1.2)
print(f"Parameters of neural network: {parameters}")

In [ ]:
X_pred = np.array([50, 120, 280])
Y_pred = predict(np.array(adv["TV"]), np.array(adv["Sales"]), parameters, X_pred)
print(f"TV marketing expenses:\n{X_pred}")
print(f"Predictions of sales:\n{Y_pred}")

#

In [ ]:
DATA_PATH2 = Path(CV_DATASETS_DIR, "csv", "house_prices_train.csv")

df = pd.read_csv(DATA_PATH2)
X_multi = df[["GrLivArea", "OverallQual"]]
Y_multi = df["SalePrice"]

display(X_multi)
display(Y_multi)

In [ ]:
X_multi_norm = (X_multi - X_multi.mean()) / X_multi.std()
Y_multi_norm = (Y_multi - Y_multi.mean()) / Y_multi.std()

# Convert results to the NumPy arrays, transpose X to get shape (2, m) and reshape Y to get (1, m)
X_multi_norm = np.array(X_multi_norm).T
Y_multi_norm = np.array(Y_multi_norm).reshape((1, len(Y_multi_norm)))

print("X_multi_norm", X_multi_norm)
print("Y_multi_norm", Y_multi_norm)

print(f"The shape of X_norm: {X_multi_norm.shape}")
print(f"The shape of Y_norm: {Y_multi_norm.shape}")

In [ ]:
parameters_multi = nn_model(X_multi_norm, Y_multi_norm, num_iterations=100, learning_rate=1.2)
print(f"Parameters of neural network: {parameters_multi}")

In [164]:
X_pred_multi = np.array([[1710, 7], [1200, 6], [2200, 8]]).T
Y_pred_multi = predict(X_multi, Y_multi, parameters_multi, X_pred_multi)

print(f"Ground living area, square feet:\n{X_pred_multi[0]}")
print(f"Rates of the overall quality of material and finish, 1-10:\n{X_pred_multi[1]}")
print(f"Predictions of sales price, $:\n{np.round(Y_pred_multi)}")

Ground living area, square feet:
[1710 1200 2200]
Rates of the overall quality of material and finish, 1-10:
[7 6 8]
Predictions of sales price, $:
[[221358. 160046. 281554.]]
